**Descarga inicial de los datos de BOJA, por disposición**

In [ ]:
#Descarga de datos por disposición

#Preparación del Entorno: Crea un directorio para almacenar los datos si no existe.
#Descarga de Datos: Utiliza una API REST para descargar datos del BOJA disposición por disposición, manejando errores y reintentando en caso de problemas temporales.
#Almacenamiento de Datos: Guarda los datos descargados en archivos JSON, organizados por año.

import requests
import json
import os
import time

# URL base para la búsqueda
base_url = "https://www.juntadeandalucia.es/ssdigitales/datasets/contentapi/search/boja.json"

# Directorio para guardar los archivos
output_folder_path = 'Data_pag'

# Asegúrate de que el directorio de salida exista, si no, créalo
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

def fetch_data(year, boja_number, disposition_number):
    query = f'disposiciones_boja-disposition.{year}.{boja_number}.{disposition_number}'
    params = {
        "q": f"_id:{query}",
        "_source": "_id,data.l_pdf.t_publicUrl,data.d_date,data.t_asumarioNoHtml,data.t_bodyNoHtml",
    }
    
    while True:
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            try:
                data = response.json()
                results = data.get("resultado", [])
                return results
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el año {year}, BOJA {boja_number}, disposición {disposition_number}: {e}")
                print(f"Respuesta del servidor: {response.content}")
                return None
        elif response.status_code == 503:
            print(f"Error 503 al solicitar los datos para el año {year}, BOJA {boja_number}, disposición {disposition_number}. Reintentando en 10 segundos...")
            time.sleep(10)
        else:
            print(f"Error {response.status_code} al solicitar los datos para el año {year}, BOJA {boja_number}, disposición {disposition_number}: {response.text}")
            time.sleep(10)

def save_year_data(year):
    all_results = []
    boja_number = 1
    max_boja_number = 1000  # Valor máximo para cubrir boletines regulares y extraordinarios

    while boja_number <= max_boja_number:
        disposition_number = 1
        boja_empty = True

        while True:
            results = fetch_data(year, boja_number, disposition_number)
            if not results:
                break
            
            for result in results:
                data = result.get("_source", {}).get("data", {})
                
                flat_result = {
                    "_id": result.get("_id"),
                    "t_publicUrl": data.get("l_pdf")[0]["t_publicUrl"] if data.get("l_pdf") else None,
                    "d_date": data.get("d_date"),
                    "t_asumarioNoHtml": data.get("t_asumarioNoHtml"),
                    "t_bodyNoHtml": data.get("t_bodyNoHtml")
                }
                all_results.append(flat_result)

            print(f"Año {year}, BOJA {boja_number}, disposición {disposition_number} descargada con {len(results)} resultados.")
            disposition_number += 1
            boja_empty = False

        if boja_empty:
            break
        boja_number += 1

    # Guardar los resultados en un archivo JSON al final del año
    output_file_path = os.path.join(output_folder_path, f"{year}.json")
    with open(output_file_path, "w", encoding='utf-8') as json_file:
        json.dump(all_results, json_file, indent=4, ensure_ascii=False)

    print(f"Se han guardado {len(all_results)} resultados en {output_file_path}")

# Bucle para obtener los resultados por año
for year in range(1987, 2025):
    save_year_data(year)

print("Descarga completada para los años 1978-2024.")




**Se ha detectado un problema en la descarga, con unas disposiciones no consecutivas, con este código se pueden descargar**

In [ ]:
#Descarga BOJA incluso sin disposiciones consecutivas (problemáticos, más lento). Error BOJA 1 de 2017
import requests
import json
import os
import time

# URL base para la búsqueda
base_url = "https://www.juntadeandalucia.es/ssdigitales/datasets/contentapi/search/boja.json"

# Directorio para guardar los archivos
output_folder_path = 'Data_pag'

# Asegúrate de que el directorio de salida exista, si no, créalo
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

def fetch_data(year, boja_number, disposition_number):
    query = f'disposiciones_boja-disposition.{year}.{boja_number}.{disposition_number}'
    params = {
        "q": f"_id:{query}",
        "_source": "_id,data.l_pdf.t_publicUrl,data.d_date,data.t_asumarioNoHtml,data.t_bodyNoHtml",
    }
    
    while True:
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            try:
                data = response.json()
                results = data.get("resultado", [])
                if not results:
                    return None
                return results
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el año {year}, BOJA {boja_number}, disposición {disposition_number}: {e}")
                print(f"Respuesta del servidor: {response.content}")
                return None
        elif response.status_code == 503:
            print(f"Error 503 al solicitar los datos para el año {year}, BOJA {boja_number}, disposición {disposition_number}. Reintentando en 10 segundos...")
            time.sleep(10)
        else:
            print(f"Error {response.status_code} al solicitar los datos para el año {year}, BOJA {boja_number}, disposición {disposition_number}: {response.text}")
            time.sleep(10)

def save_year_data(year):
    all_results = []
    boja_number = 1
    max_boja_number = 1000  # Valor máximo para cubrir boletines regulares y extraordinarios

    while boja_number <= max_boja_number:
        disposition_number = 1
        boja_empty = True
        empty_dispositions_count = 0  # Contador de disposiciones vacías

        while empty_dispositions_count < 100:  # Límite de 100 disposiciones vacías antes de pasar al siguiente BOJA
            results = fetch_data(year, boja_number, disposition_number)
            if not results:
                empty_dispositions_count += 1
                disposition_number += 1
                continue

            for result in results:
                data = result.get("_source", {}).get("data", {})
                
                flat_result = {
                    "_id": result.get("_id"),
                    "t_publicUrl": data.get("l_pdf")[0]["t_publicUrl"] if data.get("l_pdf") else None,
                    "d_date": data.get("d_date"),
                    "t_asumarioNoHtml": data.get("t_asumarioNoHtml"),
                    "t_bodyNoHtml": data.get("t_bodyNoHtml")
                }
                all_results.append(flat_result)

           
            disposition_number += 1
            boja_empty = False
            empty_dispositions_count = 0  # Resetear el contador al encontrar resultados

        if boja_empty:
            break
        boja_number += 1
        print(f"Año {year}, BOJA {boja_number}, descargado con {disposition_number} disposiciones.")

    # Guardar los resultados en un archivo JSON al final del año
    output_file_path = os.path.join(output_folder_path, f"{year}.json")
    with open(output_file_path, "w", encoding='utf-8') as json_file:
        json.dump(all_results, json_file, indent=4, ensure_ascii=False)

    print(f"Se han guardado {len(all_results)} resultados en {output_file_path}")

# Bucle para obtener los resultados por año
#for year in range(1978, 2025):


save_year_data(2017)


print("Descarga completada para los años 1978-2024.")

**Descarga de BOJAs extraordinarios, comienzan con 50X**

In [ ]:
#Descarga BOJAs extraordinarios. Posible Mejora: En ocasiones varios PDFs.

import requests
import json
import os
import time

# URL base para la búsqueda
base_url = "https://www.juntadeandalucia.es/ssdigitales/datasets/contentapi/search/boja.json"

# Directorio para guardar los archivos
output_folder_path = 'Data_pag'

# Asegúrate de que el directorio de salida exista, si no, créalo
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

def fetch_data(year, boja_number, disposition_number):
    query = f'disposiciones_boja-disposition.{year}.{boja_number}.{disposition_number}'
    params = {
        "q": f"_id:{query}",
        "_source": "_id,data.l_pdf.t_publicUrl,data.d_date,data.t_asumarioNoHtml,data.t_bodyNoHtml",
    }
    
    while True:
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            try:
                data = response.json()
                results = data.get("resultado", [])
                if not results:
                    return None
                return results
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el año {year}, BOJA {boja_number}, disposición {disposition_number}: {e}")
                print(f"Respuesta del servidor: {response.content}")
                return None
        elif response.status_code == 503:
            print(f"Error 503 al solicitar los datos para el año {year}, BOJA {boja_number}, disposición {disposition_number}. Reintentando en 10 segundos...")
            time.sleep(10)
        else:
            print(f"Error {response.status_code} al solicitar los datos para el año {year}, BOJA {boja_number}, disposición {disposition_number}: {response.text}")
            time.sleep(10)

def save_boletin_data(year, boja_number):
    all_results = []
    disposition_number = 1
    max_disposition_number = 100  # Limitar a las primeras 100 disposiciones

    while disposition_number <= max_disposition_number:
        results = fetch_data(year, boja_number, disposition_number)
        if not results:
            disposition_number += 1
            continue

        for result in results:
            data = result.get("_source", {}).get("data", {})
            
            flat_result = {
                "_id": result.get("_id"),
                "t_publicUrl": data.get("l_pdf")[0]["t_publicUrl"] if data.get("l_pdf") else None,
                "d_date": data.get("d_date"),
                "t_asumarioNoHtml": data.get("t_asumarioNoHtml"),
                "t_bodyNoHtml": data.get("t_bodyNoHtml")
            }
            all_results.append(flat_result)

        print(f"Año {year}, BOJA {boja_number}, disposición {disposition_number} descargada con {len(results)} resultados.")
        disposition_number += 1

    return all_results

def save_year_data(year):
    combined_results = []
    no_boja_found = True

    for boja_number in range(501, 700):  # Comienza con BOJA 501 y continúa hasta BOJA 599
        boja_results = save_boletin_data(year, boja_number)
        if boja_results:
            combined_results.extend(boja_results)
            no_boja_found = False
        else:
            break  # Salir del bucle si no se encuentran más disposiciones para un BOJA

    if no_boja_found:
        print(f"No se encontraron BOJA extraordinarios para el año {year}")
        return

    output_file_path = os.path.join(output_folder_path, f"{year}.json")
    
    if os.path.exists(output_file_path):
        with open(output_file_path, "r", encoding='utf-8') as json_file:
            existing_data = json.load(json_file)
        combined_results.extend(existing_data)

    # Guardar los resultados combinados en un archivo JSON
    with open(output_file_path, "w", encoding='utf-8') as json_file:
        json.dump(combined_results, json_file, indent=4, ensure_ascii=False)

    print(f"Se han guardado {len(combined_results)} resultados en {output_file_path}")

# Bucle para obtener los resultados por año
for year in range(1979, 2025):
    print(year)
    save_year_data(year)

print("Descarga completada para los años 1979-2024.")


In [ ]:
#descarga BOE. 

import os
import requests
from datetime import datetime, timedelta

boe_url = 'https://boe.es'
destino_local_raiz = 'BOE'
destino_local = os.path.join(destino_local_raiz, 'boe', 'dias')
boe_api_sumario = f'{boe_url}/diario_boe/xml.php?id=BOE-S-'

hoy = datetime.today().strftime('%Y%m%d')

def traer_documento(origen, destino):
    max_intentos = 5
    intentos = 0
    while intentos < max_intentos:
        try:
            response = requests.get(origen)
            with open(destino, 'wb') as f:
                f.write(response.content)
            return True
        except Exception as e:
            print(f"Error al descargar {origen}: {e}")
            intentos += 1
    print(f"No se pudo descargar {origen} después de {max_intentos} intentos")
    return False

if len(sys.argv) > 1:
    desde = sys.argv[1]
    hasta = sys.argv[2] if len(sys.argv) > 2 else hoy
else:
    desde = hoy
    hasta = hoy

fecha = datetime.strptime(desde, '%Y%m%d')
hasta = datetime.strptime(hasta, '%Y%m%d')

diff1Dia = timedelta(days=1)

while fecha <= hasta:
    fecha_Ymd = fecha.strftime('%Y%m%d')
    print('Fecha:', fecha_Ymd)

    fecha_anno = fecha.strftime('%Y')
    fecha_mes = fecha.strftime('%m')
    fecha_dia = fecha.strftime('%d')

    destino_local_fecha = os.path.join(destino_local, fecha_anno, fecha_mes, fecha_dia)
    os.makedirs(destino_local_fecha, exist_ok=True)
    os.makedirs(os.path.join(destino_local_fecha, 'pdfs'), exist_ok=True)

    fichero_sumario_xml = os.path.join(destino_local_fecha, 'index.xml')

    if os.path.exists(fichero_sumario_xml):
        os.remove(fichero_sumario_xml)

    print('Solicitando', boe_api_sumario + fecha_Ymd, '-->', fichero_sumario_xml)

    response = requests.get(boe_api_sumario + fecha_Ymd)
    with open(fichero_sumario_xml, 'wb') as f:
        f.write(response.content)

    tamano_sumario_xml = os.path.getsize(fichero_sumario_xml)
    print('Recibidos:', tamano_sumario_xml, 'bytes')

    if tamano_sumario_xml < 10:
        print('ERROR: Sumario XML erroneo o incompleto')
        sys.exit(1)

    # Resto del código para descargar los archivos PDF

    fecha += diff1Dia


In [ ]:
#Conversión descarga BOE a descarga BOJA

import xml.etree.ElementTree as ET
import json
import os
from collections import defaultdict

# Función para convertir la fecha al formato deseado
def convert_date(date_str):
    day, month, year = date_str.split('/')
    return f"{year}-{month}-{day}"

# Función para procesar un archivo XML del BOE y convertirlo al formato JSON del BOJA
def process_boe_xml(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Verificar si el XML contiene un error
        if root.tag == 'error':
            descripcion = root.find('descripcion').text
            print(f"Error: {descripcion} en {xml_file}")
            return []

        # Obtener metadatos
        meta = root.find('meta')
        if meta is None:
            print(f"Error: No se encontró el elemento 'meta' en {xml_file}")
            return []

        fecha = meta.find('fecha').text
        if fecha is None:
            print(f"Error: No se encontró el elemento 'fecha' en {xml_file}")
            return []

        fecha_convertida = convert_date(fecha)
        year = fecha.split('/')[-1]

        items = []
        for departamento in root.findall(".//departamento"):
            departamento_nombre = departamento.get('nombre')
            for epigrafe in departamento.findall('epigrafe'):
                epigrafe_nombre = epigrafe.get('nombre')
                for item in epigrafe.findall('item'):
                    item_id = item.get('id')
                    titulo = item.find('titulo').text
                    url_pdf = item.find('urlPdf').text
                    url_html = item.find('urlHtm').text
                    url_xml = item.find('urlXml').text

                    boe_item = {
                        "_id": item_id,
                        "t_publicUrl": f"https://www.boe.es{url_pdf}",
                        "d_date": fecha_convertida,
                        "t_asumarioNoHtml": titulo,
                        "t_bodyNoHtml": f"Departamento: {departamento_nombre}. Epígrafe: {epigrafe_nombre}.",
                        "t_urlHtm": f"https://www.boe.es{url_html}",
                        "t_urlXml": f"https://www.boe.es{url_xml}"
                    }
                    items.append((year, boe_item))

        return items
    except ET.ParseError as e:
        print(f"Error al parsear {xml_file}: {e}")
        return []

# Directorio de entrada y salida
input_dir = "BOE"
output_dir = "BOE_JSON"
os.makedirs(output_dir, exist_ok=True)

# Procesar todos los archivos XML en el directorio de entrada y agrupar por año
items_by_year = defaultdict(list)
for xml_filename in os.listdir(input_dir):
    if xml_filename.endswith('.xml'):
        xml_file_path = os.path.join(input_dir, xml_filename)
        items = process_boe_xml(xml_file_path)
        for year, item in items:
            items_by_year[year].append(item)

# Guardar los elementos en archivos JSON por año en el directorio BOE_JSON
for year, items in items_by_year.items():
    output_file_path = os.path.join(output_dir, f"boe_json_output_{year}.json")
    with open(output_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(items, json_file, ensure_ascii=False, indent=4)
    print(f"Datos del año {year} guardados en {output_file_path}")

print("Conversión completada.")


